<a href="https://colab.research.google.com/github/Rakshith-kukyan/Samarthya/blob/main/object_detection_with_R-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 6

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as vis_util
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [19]:
# Path to the input video
VIDEO_PATH = '/content/video1.mp4'

In [4]:
# Path to the frozen inference graph
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8'

In [5]:
# Load the frozen inference graph
model = tf.saved_model.load(os.path.join(PATH_TO_MODEL_DIR, 'saved_model'))

In [20]:
# Define the input and output tensors
input_tensor = model.signatures['serving_default'].inputs[0]
output_tensor = model.signatures['serving_default'].outputs

In [21]:
# Open the video file for reading
cap = cv2.VideoCapture(VIDEO_PATH)

In [22]:
# Get the video width and height
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [24]:
# Define the output video file
out = cv2.VideoWriter('/content/opvideo3.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

In [25]:
# Loop over the frames of the video
while True:
    # Read the next frame from the video
    ret, image = cap.read()
    
    # Stop the loop if we have reached the end of the video
    if not ret:
        break
    
    # Convert the image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform inference on the image
    output = model(input_tensor=tf.convert_to_tensor(image[np.newaxis, ...], dtype=tf.uint8)) # Add batch dimension

    # Extract the bounding boxes, classes, and scores from the output
    boxes = output['detection_boxes'][0].numpy()
    classes = (output['detection_classes'][0].numpy() + 1).astype(int) # Add 1 to offset the background class (0)
    scores = output['detection_scores'][0].numpy()

    # Filter out detections with low confidence scores
    min_score_thresh = 0.5
    idxs = np.where(scores > min_score_thresh)[0]
    boxes = boxes[idxs, ...]
    classes = classes[idxs]
    scores = scores[idxs]

    # Draw the detections on the image
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        boxes,
        classes,
        scores,
        {},
        use_normalized_coordinates=True,
        line_thickness=2
    )

    # Write the image with detections to the output video file
    out.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [26]:
# Release the video file and the output video file
cap.release()
out.release()